This file contains the code to find the most suitable K value for the KNN filling algorithm
Since Logistic regression performs and worst among all algorithm, we don't use it here

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn
# Cross Validation Classification Accuracy
from pandas import read_csv
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import statsmodels.api as sm
from sklearn import preprocessing 
from sklearn.impute import KNNImputer
# from sklearn.neighbors import KNeighborsClassifier
#import faiss
from numpy import isnan
#import matplotlib.pyplot as plot
from matplotlib import pyplot
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score,f1_score
from sklearn import tree
import statistics
import time
import warnings
warnings.filterwarnings('ignore')
# plotting errorbar graph for visual representation of the performance of the classifiers
import matplotlib.pyplot as plt
from matplotlib.transforms import Affine2D
import random
from xgboost import XGBClassifier
import ipynb.fs.full.our_functions_library as flib
from our_evaluate_sepsis_score import evaluate_performance
#from our_functions_library import *
import numba
from numba import jit, cuda


In [2]:
# Global initialization, get all the data from file and generate all the needed variable
filename = 'raw_data/raw_data_SetB.csv' # use raw dataset
#filename = 'data.csv' # use raw dataset
originalData = read_csv(filename) # read csv data into DataFrame var raw
print("Data size:",originalData.shape)
Original_Uniq_ID= np.unique(originalData['Patient_id']) 
Uniq_ID = Original_Uniq_ID.copy()
print('Patient id size:',len(Uniq_ID))
X_columns = ['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2','BaseExcess', 'HCO3', 'FiO2', 'pH', 
             'PaCO2', 'SaO2', 'AST', 'BUN','Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct',
             'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium','Bilirubin_total', 'TroponinI', 'Hct', 
             'Hgb', 'PTT', 'WBC','Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2','HospAdmTime',
             'ICULOS', 'Patient_id', 'time']
y_columns = ['Patient_id', 'SepsisLabel']

# To test if these 3 columns is useful
# originalData = originalData.drop(['Unit1', 'Unit2', 'EtCO2'], axis=1)
# X_columns.remove('Unit1')
# X_columns.remove( 'Unit2')
# X_columns.remove( 'EtCO2')

# Initialize the empty array X_train, X_test, y_train, y_test
X_train = pd.DataFrame(columns = X_columns)
X_test = pd.DataFrame(columns = X_columns)
y_train = pd.DataFrame(columns = y_columns)
y_test = pd.DataFrame(columns = y_columns)

# store the X & y train/test in array for easy access
globaldata = []
globaldata.append(X_train)
globaldata.append(y_train)
globaldata.append(X_test)
globaldata.append(y_test)

# Below are the lists for KNN results

KNN_UtilityScore_mean = []
KNN_UtilityScore_std = []

KNN_F1Score_mean = []
KNN_F1Score_std = []

KNN_auroc_mean = []
KNN_auprc_mean = []

KNN_accuracy_mean = []
KNN_accuracy_std = []

KNN_positiveprediction_mean = []
KNN_sepsisLabelRatio_mean = []
KNN_baseline_mean = [ ]

KNN_total_Time= [ ]


fillmethod =""  

Data size: (761995, 43)
Patient id size: 20000


In [3]:
#This block will shuffle the id sets of patients in a fix manner, so for every time you run the code, the dataset and trainset are always the same.
seed = 2
random.seed(seed)
if(np.array_equal(Original_Uniq_ID,Uniq_ID) ):
    print("They are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.")   
    random.shuffle(Uniq_ID)# randomly sorted the patient IDs    
else:
    print("They are not the same. Uniq_id has already been shuffled.")    
print("The original uniq id set is:\n",Original_Uniq_ID[1:20])
print("Uniq id:************************************")
print("The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49 345...\n",Uniq_ID[1:20])

They are the same. Uniq id hasn't been shuffled, and it will be shuffeld now.
The original uniq id set is:
 [ 2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
Uniq id:************************************
The fixed shuffelld id set, it should be 211 275 153 189 184 110 124  49 345...
 [ 2813 16575  1625  4857 11294 14365  7114  3220  2345  3642 19238 10034
 13818  5980 17451 17088 12666  3579  6276]


In [4]:
# #fill the missing data with one function
# print(filename)
# #print(originalData)
# if (originalData.isnull().values.any()):
#     print('There is data missing in the original data set')
# dataByPatient = originalData.groupby('Patient_id') # data grouped by Patient_id

In [19]:
def KNN_reset():
    #KNN with different K size, 
    KNN_UtilityScore_mean.clear()
    KNN_UtilityScore_std.clear()
    KNN_F1Score_mean.clear()
    KNN_F1Score_std.clear()
    KNN_auroc_mean.clear()
    KNN_auprc_mean.clear()
    KNN_accuracy_mean.clear()
    KNN_accuracy_std.clear()
    KNN_positiveprediction_mean.clear()
    KNN_sepsisLabelRatio_mean.clear() #array for storing the mean of the sepsislabel ratio (predict/true)
    KNN_baseline_mean.clear()
    KNN_total_Time.clear()
    
def generateTrainDataSet(test_patienIds, X_train, y_train):
    train_data= originalData[~(originalData.Patient_id.isin(test_patienIds))]
    X_train = train_data[X_columns]
    y_train = train_data[y_columns]
    return X_train, y_train
 
def generateTestDataSet(patienIds, X_test, y_test):
    #print("test_patienIds: \n", patienIds)
    test_data = originalData[originalData['Patient_id'].isin(patienIds)]
    X_test = test_data[X_columns]
    y_test = test_data[y_columns]
    return X_test, y_test

# def CalculateSOFAScore(X_train_impute, X_test_impute):
#     pass
def WritePatientIdsToFile(idSets, KforKFold):
    Twrite = time.time()
    open("test.txt", "w").close() # clear contents of existing file
    for i in range(KforKFold):
        splitted = " ".join( repr(e) for e in idSets[i])
        file1 = open("test.txt","a")
        file1.write("\n\n")
        file1.write(str("[" +splitted+"]"))
        file1.write("\n\n")
        file1.close()
    print("Time for writing id to file:", round(time.time()- Twrite,3))

In [20]:
# Code for K-fold algorithm, Before this step the missing data has been filled, 
from datetime import datetime

@jit(parallel=True)
def KFold_patient(model, KforKFold=10,KforKNN=5, fillmethod=""):
    # datetime object containing current date and time
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    print("start time =", dt_string)
    
    start = time.time()# time indicator for how long the Kfold func takes
    X_train = globaldata[0]
    y_train = globaldata[1]
    X_test = globaldata[2]
    y_test = globaldata[3]
    print("Datasize",len(Uniq_ID))
    print("K Fold of ",KforKFold ," folds with KNN =",KforKNN)
    
    # initialisation of the array for storing the different intermediate results
    accuracy_model = []
    F1Score_model = []
    baseline_model= []
    auroc_model = []
    auprc_model = []
    physio_accuracy_model = []
    f_measure_model = []
    utility_score_model = []
    mean_train = 0
    positivepredictions = []
    predictionsRatio = []
    tSplit = time.time()
    #The unique id sets have been created and shuffled in a fix manner in the third block, you can just use it here and no more any other manipulation
    idSets = np.array_split(Uniq_ID, KforKFold)# divide the ids into K groups
    print("Time for splitting id to test:", round(time.time()- tSplit,3))

    #print("Patients number: ", len(Uniq_ID)) #print('idSets arrays',idSets)
    #This for loop is for Kfold, calculating the results for K times
    WritePatientIdsToFile(idSets, KforKFold)
    
    
    for i in range(KforKFold):
        start1 = time.time()
        tClear=time.time()
        X_train, X_test, y_train, y_test = flib.clearAllDatasets(X_train, X_test, y_train, y_test) #first clear all the datasets
        print("Time for clearing X & y:", round(time.time()- tClear,4))
        
        print("for the",i+1,"th iteration: ")#,idSets[i])   
        
        tTest = time.time()
        X_test,y_test = generateTestDataSet(idSets[i], X_test, y_test)
        print("Time for splitting id to test:", round(time.time()- tTest,3))
        
        tTrain= time.time()
        X_train, y_train = generateTrainDataSet(idSets[i],X_train, y_train)
        print("Time for splitting id to train:", round(time.time()- tTrain,3))

        #Now the train and test dataset is generated
        #we can begin to train the model wit the training set and evaaulate the performance with the test sett   
#         X_train = X_train.drop('time', 1) X_test = X_test.drop('time', 1)
        X_train=X_train.astype('float64')
        X_test=X_test.astype('float64')    
        YTest_copy = y_test  # variable of joining the filled data (X) and Y (Train_output) 
        patientID_ytest = y_test['Patient_id']
        y_train = y_train.drop('Patient_id', 1)
        y_test = y_test.drop('Patient_id', 1)      
        y_train=y_train.astype('float64')
        y_test=y_test.astype('float64')
        #print('YTest',YTest_copy.head())
        
        #fill the missing data
        if X_train.isnull().values.any() or X_test.isnull().values.any() :
            tKNN= time.time()
            print("X_train or X_test contains NaN values, KNN/mean is performed.")
            #if there is missing value
            #X_train_impute, X_test_impute, fillmethod = flib.KNNfilling(X_train, X_test, KforKNN, fillmethod)
            X_train_impute, X_test_impute,fillmethod = flib.MeanFilling(X_train,X_test, fillmethod)
            #check the missing data   
            if np.isnan(X_train_impute).any() or np.isnan(X_test_impute).any() :
                print("X_train_impute or X_test_impute still contains NaN values")        
            else:
                print("X_train_impute or X_test_impute have all been filled ")
            print("Time for NaN values filling: ", round(time.time()- tKNN,3) )
        else:
            print("X_train or X_test have all been filled: ")
            X_train_impute = X_train
            X_test_impute = X_test
                 
        #Scale the data， uncomment this part for logistic regression only, in other case, put them in comment
        model_name = type(model).__name__
        if (model_name == 'LogisticRegression'):
            print("scaled!")
            scaler = preprocessing.StandardScaler()
            scaler.fit(X_train_impute)
            X_train_impute = scaler.transform(X_train_impute)
            X_test_impute = scaler.transform(X_test_impute)

        #fit the model and predict
        tModel=time.time()
        model.fit(X_train_impute, y_train)
        y_predicted = model.predict(X_test_impute)   
        y_predicted_probobility = model.predict_proba(X_test_impute) 
        #transfer the outpu and evalute it
        y_labels = y_test.astype(int).to_numpy()
        y_predicted = y_predicted.astype(int)
        if(len(y_predicted_probobility[0]) ==2 ):
            y_predicted_probobility =  y_predicted_probobility[:,1].round(4)
        else:
            print("columns of y_predicted_probobility is: ",len(y_predicted_probobility[0]))
            y_predicted_probobility =  y_predicted_probobility.round(4)
        print("Time for model fitting:", round(time.time()- tModel,3))

        tUtil=time.time()
        auroc, auprc, physio_accuracy, f_measure, utility_score = evaluate_performance(y_labels, y_predicted, y_predicted_probobility,patientID_ytest)
#         print("\nauroc",round(auroc,4),"auprc",round(auprc,4),"util_accuracy",round(accuracy1,4))
#         print("f_measure",round(f_measure,4),"utility_score",round(utility_score,4))  
        auroc_model.append(round(auroc,4))
        auprc_model.append(round(auprc,4))
        physio_accuracy_model.append(round(physio_accuracy,4))
        f_measure_model.append(round(f_measure,4))
        utility_score_model.append(round(utility_score,4))
        positivepredictions.append(np.sum(y_predicted))
        predictionsRatio.append(np.sum(y_predicted)/np.sum(y_test))
        print("Time for util calc:", round(time.time()- tUtil,4))

       
        #take down the results
        accuracy_model.append((accuracy_score(y_test, y_predicted, normalize=True)*100).round(2))
        F1Score_model.append((f1_score(y_test, y_predicted)*100).round(3))
        baseline_model.append(round( (1 - float(y_test.mean()) )*100 , 3))
        print("\ny_test size:",y_test.shape, '1´s in y_test',y_test.sum())
        #baseline_model.append(((1-y_test.mean())*100).round(2))
        print("The number of 1 (SepsisLabel) in this prediction: ", np.sum(y_predicted))
        end1 = time.time()
        print("Time spent in this KFold iteration",round(end1-start1,4),"sec.\n")
        print("******************************************************************")
    print('Accuracy model:', accuracy_model)
    print('F1_score model:', F1Score_model)
    print('Baseline model:', baseline_model)
    
    print("\nEvaluation parameters of the utiltiy evaluation function:")
    print('auroc of model:', auroc_model)
    print('auprc of model:', auprc_model)
    print('Utility accuracy of model:', accuracy_model)
    print('utility F1 of model:', f_measure_model)
    print('Utility score of model:', utility_score_model)
    
    KNN_UtilityScore_mean.append(np.mean(utility_score_model))
    KNN_UtilityScore_std.append(np.std(utility_score_model))
    
    KNN_F1Score_mean.append(np.mean(f_measure_model))
    KNN_F1Score_std.append(np.std(f_measure_model))
    
    KNN_auroc_mean.append(np.mean(auroc_model))
    KNN_auprc_mean.append(np.mean(auprc_model))
    
    KNN_accuracy_mean.append(np.mean(physio_accuracy_model))
    KNN_accuracy_std.append(np.std(physio_accuracy_model))
    
    KNN_positiveprediction_mean.append(np.mean(positivepredictions))
    KNN_sepsisLabelRatio_mean.append(np.mean(predictionsRatio))
    
    KNN_baseline_mean.append(np.mean(baseline_model))
    
    totalTime=round(time.time()- start,3)
    KNN_total_Time.append(totalTime)
    
    print("\nTotal Time spent in  KFold function",totalTime,"sec.\n")


In [21]:
def Print_DATA():
    print("KNN_UtilityScore_mean", KNN_UtilityScore_mean)
    print("KNN_UtilityScore_std", KNN_UtilityScore_std)
    print("KNN_F1Score_mean", KNN_F1Score_mean)
    print("KNN_F1Score_std", KNN_F1Score_std)
    print("KNN_auroc_mean", KNN_auroc_mean)
    print("KNN_auprc_mean", KNN_auprc_mean)
    print("KNN_accuracy_mean", KNN_accuracy_mean)
    print("KNN_accuracy_std", KNN_accuracy_std)
    print("KNN_positiveprediction_mean", KNN_positiveprediction_mean)
    print("KNN_sepsisLabelRatio_mean", KNN_sepsisLabelRatio_mean)
    print("KNN_baseline_mean", KNN_baseline_mean)
    print("KNN_total_time", KNN_total_Time)

In [22]:
#Here is how to train the model
#logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='saga', max_iter=1000,penalty='l1')
logisticRegressionModel = LogisticRegression(C=0.1,random_state=2,solver='lbfgs', max_iter=1000)
decisionTreeModel = tree.DecisionTreeClassifier(random_state=2)
randomForestModel = RandomForestClassifier(random_state=2)
XGBoostModel = XGBClassifier(random_state=2, eval_metric='error')
AdaBoostModel = AdaBoostClassifier(random_state=2)

In [9]:
#fill the missing data with one function
print(filename)
#print(originalData)
if (originalData.isnull().values.any()):
    print('Data isn´t all filled before K-Fold Func')
    originalData1=originalData.copy()
originalData,fillmethod = flib.linearFillingAll(originalData,1, True)#forwardfilling + fill other missing data with 0
#dataByPatient = data.groupby('Patient_id') # data grouped by Patient_id


raw_data/raw_data_SetB.csv
Data isn´t all filled before K-Fold Func
filling using Forward filling
The filling choice is of number: 1
fillingChoice =1
total time: 5934.27 sec


In [10]:
originalData_copy = originalData.copy()
originalData_copy.head()
originalData_copy.to_csv(r'./SetB_ffil.csv', index=False, header=True)

## FFILL with decisionTree model

In [25]:
KNN_reset()
KFold_patient(decisionTreeModel,10,5)
print("\nDecisionTreeModel for K = 350\n")
KFold_patient(decisionTreeModel,10,350)

start time = 20/03/2021 23:08:13
Datasize 20000
K Fold of  10  folds with KNN = 5
Time for splitting id to test: 0.0
Time for writing id to file: 0.027
Time for clearing X & y: 0.0001
for the 1 th iteration: 
Time for splitting id to test: 0.074
Time for splitting id to train: 0.515
X_train or X_test have all been filled: 
Time for model fitting: 62.708
Time for util calc: 2.7376

y_test size: (77277, 1) 1´s in y_test SepsisLabel    1141.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  2360
Time spent in this KFold iteration 66.5369 sec.

******************************************************************
Time for clearing X & y: 0.0003
for the 2 th iteration: 
Time for splitting id to test: 0.046
Time for splitting id to train: 0.562
X_train or X_test have all been filled: 
Time for model fitting: 65.48
Time for util calc: 3.9423

y_test size: (76043, 1) 1´s in y_test SepsisLabel    1015.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  2303
Time sp

Time for splitting id to train: 0.383
X_train or X_test have all been filled: 
Time for model fitting: 64.588
Time for util calc: 2.6699

y_test size: (76004, 1) 1´s in y_test SepsisLabel    1159.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  2105
Time spent in this KFold iteration 68.0219 sec.

******************************************************************
Time for clearing X & y: 0.0003
for the 7 th iteration: 
Time for splitting id to test: 0.05
Time for splitting id to train: 0.362
X_train or X_test have all been filled: 
Time for model fitting: 66.845
Time for util calc: 2.6726

y_test size: (74674, 1) 1´s in y_test SepsisLabel    1078.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  1884
Time spent in this KFold iteration 70.2414 sec.

******************************************************************
Time for clearing X & y: 0.0003
for the 8 th iteration: 
Time for splitting id to test: 0.042
Time for splitting id to train: 0.357
X_tra

In [26]:
Print_DATA()

KNN_UtilityScore_mean [0.08847000000000002, 0.08847000000000002]
KNN_UtilityScore_std [0.0243145244658414, 0.0243145244658414]
KNN_F1Score_mean [0.09775, 0.09775]
KNN_F1Score_std [0.015516910130564011, 0.015516910130564011]
KNN_auroc_mean [0.0, 0.0]
KNN_auprc_mean [0.0, 0.0]
KNN_accuracy_mean [0.96405, 0.96405]
KNN_accuracy_std [0.0030229952034364653, 0.0030229952034364653]
KNN_positiveprediction_mean [1959.6, 1959.6]
KNN_sepsisLabelRatio_mean [1.836271942793846, 1.836271942793846]
KNN_baseline_mean [98.5843, 98.5843]
KNN_total_time [682.575, 638.471]


## FFILL with XGBOOST model

In [27]:
KNN_reset()
KFold_patient(XGBoostModel,10,5)
print("\nXGBoostModel for K = 350\n")
KFold_patient(XGBoostModel,10,350)

start time = 20/03/2021 23:30:14
Datasize 20000
K Fold of  10  folds with KNN = 5
Time for splitting id to test: 0.0
Time for writing id to file: 0.044
Time for clearing X & y: 0.0
for the 1 th iteration: 
Time for splitting id to test: 0.099
Time for splitting id to train: 0.513
X_train or X_test have all been filled: 
Time for model fitting: 129.113
Time for util calc: 3.4066

y_test size: (77277, 1) 1´s in y_test SepsisLabel    1141.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  309
Time spent in this KFold iteration 133.6133 sec.

******************************************************************
Time for clearing X & y: 0.0003
for the 2 th iteration: 
Time for splitting id to test: 0.043
Time for splitting id to train: 0.353
X_train or X_test have all been filled: 
Time for model fitting: 129.831
Time for util calc: 3.3176

y_test size: (76043, 1) 1´s in y_test SepsisLabel    1015.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  315
Time spe

Time for splitting id to train: 0.347
X_train or X_test have all been filled: 
Time for model fitting: 129.032
Time for util calc: 3.3363

y_test size: (76004, 1) 1´s in y_test SepsisLabel    1159.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  222
Time spent in this KFold iteration 133.0858 sec.

******************************************************************
Time for clearing X & y: 0.0003
for the 7 th iteration: 
Time for splitting id to test: 0.043
Time for splitting id to train: 0.371
X_train or X_test have all been filled: 
Time for model fitting: 127.571
Time for util calc: 3.2936

y_test size: (74674, 1) 1´s in y_test SepsisLabel    1078.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  293
Time spent in this KFold iteration 131.5976 sec.

******************************************************************
Time for clearing X & y: 0.0003
for the 8 th iteration: 
Time for splitting id to test: 0.044
Time for splitting id to train: 0.352
X_

In [28]:
Print_DATA()

KNN_UtilityScore_mean [0.08666, 0.08666]
KNN_UtilityScore_std [0.027802165383293435, 0.027802165383293435]
KNN_F1Score_mean [0.14295, 0.14295]
KNN_F1Score_std [0.035797856081056026, 0.035797856081056026]
KNN_auroc_mean [0.85512, 0.85512]
KNN_auprc_mean [0.13987999999999998, 0.13987999999999998]
KNN_accuracy_mean [0.98424, 0.98424]
KNN_accuracy_std [0.001182539639927576, 0.001182539639927576]
KNN_positiveprediction_mean [321.4, 321.4]
KNN_sepsisLabelRatio_mean [0.3039322346837608, 0.3039322346837608]
KNN_baseline_mean [98.5843, 98.5843]
KNN_total_time [1330.018, 1321.152]


## FFILL with ADABOOST model

In [29]:
KNN_reset()
KFold_patient(AdaBoostModel,10,5)
print("\nAdaBoostModel for K = 350\n")
KFold_patient(AdaBoostModel,10,350)

start time = 21/03/2021 00:14:25
Datasize 20000
K Fold of  10  folds with KNN = 5
Time for splitting id to test: 0.0
Time for writing id to file: 0.035
Time for clearing X & y: 0.0
for the 1 th iteration: 
Time for splitting id to test: 0.047
Time for splitting id to train: 0.348
X_train or X_test have all been filled: 
Time for model fitting: 147.598
Time for util calc: 2.9553

y_test size: (77277, 1) 1´s in y_test SepsisLabel    1141.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  136
Time spent in this KFold iteration 151.2729 sec.

******************************************************************
Time for clearing X & y: 0.0003
for the 2 th iteration: 
Time for splitting id to test: 0.049
Time for splitting id to train: 0.358
X_train or X_test have all been filled: 
Time for model fitting: 149.953
Time for util calc: 3.612

y_test size: (76043, 1) 1´s in y_test SepsisLabel    1015.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  212
Time spen

Time for splitting id to train: 0.355
X_train or X_test have all been filled: 
Time for model fitting: 149.736
Time for util calc: 3.5391

y_test size: (76004, 1) 1´s in y_test SepsisLabel    1159.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  153
Time spent in this KFold iteration 153.9924 sec.

******************************************************************
Time for clearing X & y: 0.0003
for the 7 th iteration: 
Time for splitting id to test: 0.043
Time for splitting id to train: 0.37
X_train or X_test have all been filled: 
Time for model fitting: 149.764
Time for util calc: 2.9207

y_test size: (74674, 1) 1´s in y_test SepsisLabel    1078.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  132
Time spent in this KFold iteration 153.4374 sec.

******************************************************************
Time for clearing X & y: 0.0003
for the 8 th iteration: 
Time for splitting id to test: 0.049
Time for splitting id to train: 0.349
X_t

In [30]:
Print_DATA()

KNN_UtilityScore_mean [0.032630000000000006, 0.032630000000000006]
KNN_UtilityScore_std [0.013634005280914336, 0.013634005280914336]
KNN_F1Score_mean [0.06079, 0.06079]
KNN_F1Score_std [0.02300262811071813, 0.02300262811071813]
KNN_auroc_mean [0.8471299999999999, 0.8471299999999999]
KNN_auprc_mean [0.1069, 0.1069]
KNN_accuracy_mean [0.98472, 0.98472]
KNN_accuracy_std [0.0013481839637082106, 0.0013481839637082106]
KNN_positiveprediction_mean [160.5, 160.5]
KNN_sepsisLabelRatio_mean [0.1511054252384753, 0.1511054252384753]
KNN_baseline_mean [98.5843, 98.5843]
KNN_total_time [1534.68, 1528.007]


## linear fill on logistic regression - 

In [32]:
KNN_reset()
KFold_patient(logisticRegressionModel,10,5)
print("\nAdaboost for K = 350\n")
KFold_patient(logisticRegressionModel,10,350)

start time = 21/03/2021 18:33:35
Datasize 20000
K Fold of  10  folds with KNN = 5
Time for splitting id to test: 0.0
Time for writing id to file: 0.05
Time for clearing X & y: 0.0
for the 1 th iteration: 
Time for splitting id to test: 0.071
Time for splitting id to train: 0.376
X_train or X_test have all been filled: 
scaled!
Time for model fitting: 10.657
Time for util calc: 2.9805

y_test size: (77277, 1) 1´s in y_test SepsisLabel    1141.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  11
Time spent in this KFold iteration 16.4256 sec.

******************************************************************
Time for clearing X & y: 0.0003
for the 2 th iteration: 
Time for splitting id to test: 0.042
Time for splitting id to train: 0.362
X_train or X_test have all been filled: 
scaled!
Time for model fitting: 16.105
Time for util calc: 2.9382

y_test size: (76043, 1) 1´s in y_test SepsisLabel    1015.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  0

Time for splitting id to train: 0.375
X_train or X_test have all been filled: 
scaled!
Time for model fitting: 16.492
Time for util calc: 3.0968

y_test size: (76004, 1) 1´s in y_test SepsisLabel    1159.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  34
Time spent in this KFold iteration 22.3069 sec.

******************************************************************
Time for clearing X & y: 0.0004
for the 7 th iteration: 
Time for splitting id to test: 0.057
Time for splitting id to train: 0.543
X_train or X_test have all been filled: 
scaled!
Time for model fitting: 18.538
Time for util calc: 2.8643

y_test size: (74674, 1) 1´s in y_test SepsisLabel    1078.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  8
Time spent in this KFold iteration 24.4182 sec.

******************************************************************
Time for clearing X & y: 0.0003
for the 8 th iteration: 
Time for splitting id to test: 0.047
Time for splitting id to train:

In [33]:
Print_DATA()

KNN_UtilityScore_mean [0.00162, 0.00162]
KNN_UtilityScore_std [0.0036950778070292374, 0.0036950778070292374]
KNN_F1Score_mean [0.0039700000000000004, 0.0039700000000000004]
KNN_F1Score_std [0.007509201022745361, 0.007509201022745361]
KNN_auroc_mean [0.78682, 0.78682]
KNN_auprc_mean [0.06819, 0.06819]
KNN_accuracy_mean [0.98566, 0.98566]
KNN_accuracy_std [0.001116422858956238, 0.001116422858956238]
KNN_positiveprediction_mean [20.5, 20.5]
KNN_sepsisLabelRatio_mean [0.01962473032540857, 0.01962473032540857]
KNN_baseline_mean [98.5843, 98.5843]
KNN_total_time [225.997, 235.127]


## Best Utiltity score for best K of 400, 1000 but now applied on 2000 patient dataset

In [18]:
# print("Logistic Regression")
KFold_patient(decisionTreeModel,10, 325)

K Fold of  10  folds with KNN = 325
for the 1 th iteration: 
Time for splitting id to test: 2.09
Time for splitting id to train: 130.86
X_train or X_test contains NaN values, KNN/mean is performed.
X_train_impute or X_test_impute have all been filled 
Time for NaN values filling: 

y_test size: (8053, 1) 1´s in y_test SepsisLabel    144.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  500
Time spent in this KFold iteration 2291.18 sec.

******************************************************************
for the 2 th iteration: 
Time for splitting id to test: 1.28
Time for splitting id to train: 43.63
X_train or X_test contains NaN values, KNN/mean is performed.
X_train_impute or X_test_impute have all been filled 
Time for NaN values filling: 

y_test size: (8176, 1) 1´s in y_test SepsisLabel    176.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  488
Time spent in this KFold iteration 2068.81 sec.

**************************************************

In [23]:
Print_DATA()

KNN_UtilityScore_mean [0.1271]
KNN_UtilityScore_std [0.07827688036706625]
KNN_F1Score_mean [0.10593]
KNN_F1Score_std [0.038923413262456824]
KNN_auroc_mean [0.0]
KNN_auprc_mean [0.0]
KNN_accuracy_mean [0.9439]
KNN_accuracy_std [0.01649120977975842]
KNN_positiveprediction_mean [351.4]
KNN_baseline_mean [98.162]
KNN_total_time [20363.62]


In [24]:
KFold_patient(decisionTreeModel,10, 210)

K Fold of  10  folds with KNN = 210
for the 1 th iteration: 
Time for splitting id to test: 1.56
Time for splitting id to train: 48.89
X_train or X_test contains NaN values, KNN/mean is performed.
X_train_impute or X_test_impute have all been filled 
Time for NaN values filling:  1980.97

y_test size: (8053, 1) 1´s in y_test SepsisLabel    144.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  369
Time spent in this KFold iteration 2037.29 sec.

******************************************************************
for the 2 th iteration: 
Time for splitting id to test: 1.38
Time for splitting id to train: 40.86
X_train or X_test contains NaN values, KNN/mean is performed.
X_train_impute or X_test_impute have all been filled 
Time for NaN values filling:  1907.43

y_test size: (8176, 1) 1´s in y_test SepsisLabel    176.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  366
Time spent in this KFold iteration 1954.72 sec.

***********************************

In [25]:
Print_DATA_DATA()

KNN_UtilityScore_mean [0.1271, 0.13489]
KNN_UtilityScore_std [0.07827688036706625, 0.07207767268717823]
KNN_F1Score_mean [0.10593, 0.12088000000000002]
KNN_F1Score_std [0.038923413262456824, 0.0449781902703966]
KNN_auroc_mean [0.0, 0.0]
KNN_auprc_mean [0.0, 0.0]
KNN_accuracy_mean [0.9439, 0.9514700000000001]
KNN_accuracy_std [0.01649120977975842, 0.010457346699808715]
KNN_positiveprediction_mean [351.4, 289.1]
KNN_baseline_mean [98.162, 98.162]
KNN_total_time [20363.62, 18964.78]


In [26]:
KFold_patient(XGBoostModel,10, 355)

K Fold of  10  folds with KNN = 355
for the 1 th iteration: 
Time for splitting id to test: 1.13
Time for splitting id to train: 40.68
X_train or X_test contains NaN values, KNN/mean is performed.
X_train_impute or X_test_impute have all been filled 
Time for NaN values filling:  1875.17

y_test size: (8053, 1) 1´s in y_test SepsisLabel    144.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  55
Time spent in this KFold iteration 1930.61 sec.

******************************************************************
for the 2 th iteration: 
Time for splitting id to test: 1.07
Time for splitting id to train: 40.68
X_train or X_test contains NaN values, KNN/mean is performed.
X_train_impute or X_test_impute have all been filled 
Time for NaN values filling:  1919.27

y_test size: (8176, 1) 1´s in y_test SepsisLabel    176.0
dtype: float64
The number of 1 (SepsisLabel) in this prediction:  191
Time spent in this KFold iteration 1973.29 sec.

************************************

In [27]:
Print_DATA()

KNN_UtilityScore_mean [0.1271, 0.13489, 0.08547]
KNN_UtilityScore_std [0.07827688036706625, 0.07207767268717823, 0.06969251107543765]
KNN_F1Score_mean [0.10593, 0.12088000000000002, 0.10774999999999998]
KNN_F1Score_std [0.038923413262456824, 0.0449781902703966, 0.06448243559295819]
KNN_auroc_mean [0.0, 0.0, 0.7326600000000001]
KNN_auprc_mean [0.0, 0.0, 0.09212000000000001]
KNN_accuracy_mean [0.9439, 0.9514700000000001, 0.97098]
KNN_accuracy_std [0.01649120977975842, 0.010457346699808715, 0.0093056756874501]
KNN_positiveprediction_mean [351.4, 289.1, 113.6]
KNN_baseline_mean [98.162, 98.162, 98.162]
KNN_total_time [20363.62, 18964.78, 19595.06]


In [35]:
# Accuracy model: [94.39, 89.65, 77.34, 83.25, 93.85, 93.94, 88.73, 92.87, 97.38, 91.34]
# F1_score model: [0.0, 2.53, 5.58, 2.94, 0.0, 0.0, 18.75, 24.66, 0.0, 11.11]
# Baseline model: [98.47, 97.58, 98.21, 95.94, 98.42, 98.48, 95.81, 95.2, 98.54, 94.59]

# Evaluation parameters of the utiltiy evaluation function:
# auroc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# auprc of model: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# Utility accuracy of model: [94.39, 89.65, 77.34, 83.25, 93.85, 93.94, 88.73, 92.87, 97.38, 91.34]
# utility F1 of model: [0.0, 0.0253, 0.0558, 0.0294, 0.0, 0.0, 0.1875, 0.2466, 0.0, 0.1111]
# Utility score of model: [-0.0923, -0.0583, 0.0301, -0.0208, -0.0837, -0.0857, 0.2554, 0.2509, -0.0229, 0.0562]

# Total Time spent in  KFold function 109.83 sec.

In [14]:
# print("Decision tree")
#KFold_patient(randomForestModel,10, 2)

In [15]:
# findBestKforKNN(logisticRegressionModel,10,1,10)
# findBestKforKNN(XGBoostModel,10,110,120)

In [16]:
# findBestKforKNN(decisionTreeModel,10,1,10)
# os.system('\a')

In [17]:
displayCurrentResult(1,9)

NameError: name 'displayCurrentResult' is not defined